In [1]:
import pandas as pd
import numpy as np

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    data['DOLocationID'] = data['DOLocationID'].astype(str)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
hub = 'Lga'
tune_hyp_params = False

In [5]:
dataDir = '/home/urwa/Documents/side_projects/urban/data/processedData/'
file = dataDir + hub + 'VehiceByHour.csv'

In [6]:
rawdata = loadData(file)

Raw shape:  (2251320, 4)
Days:  365


In [7]:
rawdata.head(2)

,DOLocationID,Date,Hour,vehicle_count
0,1,2018-01-01,0,0.0
1,2,2018-01-01,0,0.0


In [8]:
edge_data = getTimeSeries(rawdata)
edge_data = edge_data.reset_index()
edge_data.head(3)

DOLocationID,Date,Hour,1,10,100,101,102,106,107,108,...,90,91,92,93,94,95,96,97,98,99
0,2018-01-01,0,0,0,0,0,0,2,1,0,...,0,0,1,0,1,0,0,1,0,0
1,2018-01-01,1,0,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,2018-01-01,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Merge external dataset

In [9]:
externalDataDir = "/home/urwa/Documents/side_projects/urban/data/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [10]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,3,89,67,156,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,18/1/1 1:00,0,17,8,25,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [11]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,3,89,67,156,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,0,17,8,25,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [12]:
min(extDf.date), max(extDf.date)

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [13]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [14]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [15]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [16]:
extDf = extDf[selected_columns]

In [17]:
print(edge_data.shape)
print(extDf.shape)

(8760, 259)
(8760, 14)


In [18]:
edge_data['Date'] = pd.to_datetime(edge_data['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [19]:
edge_data = pd.merge(edge_data,extDf, on=['Date', 'Hour'], how='inner')
print(edge_data.shape)
edge_data['Date'] = edge_data['Date'].dt.date
edge_data.head()

(8760, 271)


,Date,Hour,1,10,100,101,102,106,107,108,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01,0,0,0,0,0,0,2,1,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
1,2018-01-01,1,0,0,1,1,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
2,2018-01-01,2,0,0,0,0,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
3,2018-01-01,3,0,0,0,0,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0
4,2018-01-01,4,1,0,0,0,0,0,0,0,...,19,8,13.5,-20.5,51,0,0.0,0.0,0,0


In [20]:
edge_data.columns

Index(['Date', 'Hour', '1', '10', '100', '101', '102', '106', '107', '108',
       ...
       'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd', 'cdd',
       'participation', 'newsnow', 'snowdepth', 'ifSnow'],
      dtype='object', length=271)

In [21]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [22]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in edge_data.columns if c not in ext_columns and \
                c not in DateColumns and c != 'Hour']

lagColumns = targetColumns + ['arrival']

In [23]:
maxlag = 3

edge_data = addLag(edge_data, maxlag, lagColumns)

edge_data.shape

(8757, 1045)

In [24]:
edge_data.to_csv('/home/urwa/Documents/side_projects/urban/data/featureData/lga.csv', index=False)